In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('data/gemstone.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [3]:
df=df.drop(labels=['id'],axis=1)

In [4]:
df.head(2)

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387


In [5]:
#dependent and independent varibale 
x=df.drop(labels=['price'],axis=1)
y=df[['price']]

In [6]:
y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [7]:
#segregate categoriacal and numerical olumns
categorical_cols=x.select_dtypes(include='object').columns
numerical_cols=x.select_dtypes(exclude='object').columns

In [8]:
categorical_cols

Index(['cut', 'color', 'clarity'], dtype='object')

In [9]:
numerical_cols

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [35]:
#Define the custom ranking for each ordinal varibale
cut_categories=['Fair','Good','Very Good','Premium','Ideal']
color_categories=['D','E','F','G','H','I','J']
clarity_categories=['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [36]:
#import all required modules
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

In [37]:
#Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [38]:
#Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
('imputer',SimpleImputer(strategy='median')),

('scaler',StandardScaler())
    ]
)

# categorical Pipeline
cat_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinal encoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())
    ]
)
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
    ])

In [39]:
# train test split
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=30)

In [40]:
x_train = pd.DataFrame(preprocessor.fit_transform(x_train), columns=preprocessor.get_feature_names_out())
x_test = pd.DataFrame(preprocessor.transform(x_test), columns=preprocessor.get_feature_names_out())


In [41]:
# model training
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

regression=LinearRegression()
regression.fit(x_train,y_train)

regression.coef_
regression.intercept_



array([3970.76628955])

In [42]:
regression.coef_

array([[ 6433.66003594,  -132.75843566,   -70.42922179, -1720.30971463,
         -499.29302619,   -63.39317848,    72.44537247,  -460.41604642,
          650.76431652]])

In [43]:
regression.intercept_

array([3970.76628955])

In [44]:
import numpy as np
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square=r2_score(true,predicted)
    return mae,mse,rmse,r2_square

In [56]:
#Train multiple model

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet()
}

trained_model_list=[]
model_list=[]
r2_list=[]

for i in range (len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)
    
    #make predication
    y_pred=model.predict(x_test)
    mae,mse,rmse,r2_square = evaluate_model(y_test,y_pred)
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model Training Performance')
    print('RMSE',rmse)
    print('MAE',mae)
    print('R2_Score',r2_square*100)
    
    r2_list.append(r2_square)
    
    print("="*35)
    print('\n')


LinearRegression
Model Training Performance
RMSE 1013.9047094344005
MAE 674.0255115796832
R2_Score 93.68908248567512


Lasso
Model Training Performance
RMSE 1013.8784226767013
MAE 675.0716923362161
R2_Score 93.68940971841704


Ridge
Model Training Performance
RMSE 1013.9059272771568
MAE 674.0555800798383
R2_Score 93.68906732505947


ElasticNet
Model Training Performance
RMSE 1533.4162456064048
MAE 1060.7368759154729
R2_Score 85.56494831165182




In [57]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'ElasticNet']